In [1]:
import pandas as pd
import os

In [2]:
# Define source path and files
source_path = 'Resources'
source_csv_files = [os.path.join(source_path, file) for file in os.listdir(source_path) if file.endswith('.csv')]

In [3]:
# Create a list to store all the temporary dataframes
temp_df = []
common_column = 'DDM Sample ID'

In [4]:
# Loop through the csv files appending as dataframes to temporary dataframes list
for file in source_csv_files:
    temp_df.append(pd.read_csv(file))

In [5]:
# Merge dataframes with first dataframe on Lab number
merged_data_df = temp_df[0]

for df in temp_df[1:]:
    merged_data_df = pd.merge(merged_data_df, df, on=common_column, how='inner')

In [6]:
# Get all the column names as a list
all_columns = merged_data_df.columns.to_list()

# Save all the column names to a text file for editing
with open('all_columns.txt', "w") as file:
    for item in all_columns:
        file.write(str(item) + "\n")

# Save the merged dataframes to a csv to check
merged_data_df.to_csv('temp_merged.csv', index=False)

In [7]:
# Get columns to keep as a list
with open("keep_columns.txt", "r") as file:
    keep_columns = [line.strip() for line in file]

# Get new column names as a list
with open("new_column_names.txt", "r") as file:
    renamed_columns = [line.strip() for line in file]

# Zip lists together to make column renaming dictionary
new_columns_dict = dict(zip(keep_columns, renamed_columns))

In [8]:
# Drop the columns not in the 'keep_columns' list
drop_columns = merged_data_df.columns[~merged_data_df.columns.isin(keep_columns)]
merged_data_df = merged_data_df.drop(columns=drop_columns)
merged_data_df.head()

,DDM Sample ID,MID-S,MonthsOld,Source_x,WGS (M)_x,Res Noise,Purity,SOPHiA_GI_status,SOPHiA_GI_index,Myriad_GI_score_x,...,100x,200x,500x,1000x,Run,Episode#,Panel (M),Detected Gene,Detected Variant,Detected VAF (%)
0,200058320-107-S2,2,47,AZ,5.9,0.13,20,1,3.2,51,...,84%,49%,5%,1%,1,12749097,1.4,.,.,.
1,200058326-106-S3,3,93,AZ,5.6,0.11,30,2,-15.7,20,...,97%,73%,12%,2%,1,12749205,1.7,.,.,.
2,200058327-104-S4,4,71,AZ,6.1,0.1,20,2,-4.6,17,...,100%,100%,97%,41%,1,12749267,3.5,.,.,.
3,200058329-109-S6,6,167,AZ,5.6,0.09,20,2,-4.6,29,...,100%,99%,71%,16%,1,12749335,3.3,.,.,.
4,200058330-91-S7,7,131,AZ,5.0,0.11,60,2,-8.2,29,...,100%,100%,46%,2%,1,12749366,3.6,.,.,.


In [9]:
# Rename and re-order columns
merged_data_df = merged_data_df.rename(columns=new_columns_dict)
merged_data_df= merged_data_df.reindex(columns=new_columns_dict.values())
merged_data_df.head()

,Run,SampleID,Source,MonthsOld,Purity,SeqRunID,DDMSampleID,MIDS,TotalReads(M),lpWGSReads(M),...,ResNoise,SignalNoiseRatio,QAStatus,Gene,Variant,%VariantFraction,MyriadGIScore,MyriadGIStatus,SOPHiAGIIndex,SophiaGIStatus
0,1,12749097,AZ,47,20,220121_NB501056_0748_AH2CV5BGXK,200058320-107-S2,2,7.3,5.9,...,0.13,2.95,Medium,.,.,.,51,1,3.2,1
1,1,12749205,AZ,93,30,220121_NB501056_0748_AH2CV5BGXK,200058326-106-S3,3,7.3,5.6,...,0.11,2.91,High,.,.,.,20,2,-15.7,2
2,1,12749267,AZ,71,20,220121_NB501056_0748_AH2CV5BGXK,200058327-104-S4,4,9.6,6.1,...,0.1,1.64,High,.,.,.,17,2,-4.6,2
3,1,12749335,AZ,167,20,220121_NB501056_0748_AH2CV5BGXK,200058329-109-S6,6,8.9,5.6,...,0.09,3.49,High,.,.,.,29,2,-4.6,2
4,1,12749366,AZ,131,60,220121_NB501056_0748_AH2CV5BGXK,200058330-91-S7,7,8.6,5.0,...,0.11,2.18,High,.,.,.,29,2,-8.2,2


In [10]:
# Save final dataframe to csv
output_filename = 'PeterMac_HRD_Validation.csv'
output_file_path = os.path.join(source_path, output_filename)
merged_data_df.to_csv(output_file_path, index=False)